In [176]:
import numpy as np
import pandas as pd

%matplotlib inline

## Params

In [206]:
lgb_submit = pd.read_csv('./data/submission/lgb_estim_validation.csv')
tf_submit = pd.read_csv('./data/submission/tf_estim_validation.csv')
prophet_top_down_submit = pd.read_csv('./data/submission/Prophet_store_dpt_lgb_weights_validation.csv')
wavenet_top_down_submit = pd.read_csv('./data/submission/Wavenet_store_dpt_lgb_weights_validation.csv')
lgb_top_down_submit = pd.read_csv('./data/submission/Lgb_store_dpt_lgb_weights_validation.csv')

for c in range(1,29):
    tf_submit['F%s' % c].clip(0, inplace=True)

In [207]:
true = pd.read_csv('data/raw/sales_train_evaluation.csv')[['id'] + ['d_%s' % c for c in range(1914, 1942)]]
true.columns = ['id'] + ['F%s' % c for c in range(1, 29)]
true['id'] = true['id'].str.replace('evaluation', 'validation')

In [210]:
import functools

errors = functools.reduce(pd.merge,
                 [
                     pd.melt(lgb_submit.query('id.str.contains("validation")'), id_vars='id', value_name='lgb'),
                     pd.melt(tf_submit.query('id.str.contains("validation")'), id_vars='id', value_name='tf'),
                     pd.melt(prophet_top_down_submit.query('id.str.contains("validation")'), id_vars='id', value_name='prophet_top_down'),
                     pd.melt(wavenet_top_down_submit.query('id.str.contains("validation")'), id_vars='id', value_name='wavenet_top_down'),
                     pd.melt(lgb_top_down_submit.query('id.str.contains("validation")'), id_vars='id', value_name='lgb_top_down'),
                     pd.melt(true, id_vars='id', value_name='true')
                 ]
                )

errors['neutral'] = 0

In [212]:
errors['store_id'] = errors['id'].apply(lambda x: '_'.join(x.split('_')[3:5]))
errors['dept_id'] = errors['id'].apply(lambda x: '_'.join(x.split('_')[0:2]))

In [213]:
list_ensemble = list()
for _, gp in errors.groupby(['store_id', 'dept_id']):
    errors_gp = list()
    for m in ['lgb', 'tf', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down', 'neutral']:
        errors_gp.append(np.sqrt(((gp['true']-gp[m])**2).mean()))
    
    es = errors_gp[:5]
    ps = gp[['lgb', 'tf', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down']].values
    e0 = errors_gp[-1]
    l=0.0001
    m = len(es)
    n = len(ps)
    X = ps
    pTy = .5 * (n * e0**2 + (X**2).sum(axis=0) - n * np.array(es)**2)
    w = np.linalg.pinv(X.T.dot(X) + l * n * np.eye(m)).dot(pTy)
    weights_dict[_] = w
    
    gp['ensemble'] = gp[['lgb', 'tf', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down']].values.dot(w)
    list_ensemble.append(gp)

/home/dataexmachina/DataProjects/m5-forts-castors/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [216]:
ensemble = pd.concat(list_ensemble)
for m in ['lgb', 'tf', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down', 'ensemble']:
    print(m, np.sqrt(((ensemble['true']-ensemble[m])**2).mean()))

lgb 2.0887245479323964
tf 2.0980360379853136
prophet_top_down 2.113325014023945
wavenet_top_down 2.118072537261426
lgb_top_down 2.1268339606220357
ensemble 2.063142692823469


In [217]:
ensemble['ensemble'].clip(0, inplace=True)

In [218]:
ensemble = ensemble.pivot_table(index=['id'], columns=['variable'], values=['ensemble']).reset_index()
ensemble.columns = [c[0] if c[1]=='' else c[1] for c in ensemble.columns.tolist()]
ensemble = ensemble[['id'] + ['F%s' % c for c in range(1,29)]]

In [219]:
ensemble

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.826237,0.767981,0.748609,0.748718,1.041785,0.960593,0.940726,0.841558,0.881715,...,0.938737,1.172384,1.007473,0.759950,0.751460,0.691634,0.726349,0.915185,0.991758,0.956008
1,FOODS_1_001_CA_2_validation,1.115632,1.136502,1.125024,1.435687,1.537009,1.422954,1.827383,1.070285,1.099661,...,1.429203,1.793364,1.871851,1.332136,1.259438,1.252809,1.281068,1.490080,1.962740,1.661314
2,FOODS_1_001_CA_3_validation,1.462551,1.518316,1.401017,1.389065,1.519076,0.903820,1.006145,1.285441,1.462079,...,1.393806,1.704460,1.475626,1.254162,1.233328,1.180875,1.194592,1.350658,1.524805,1.268716
3,FOODS_1_001_CA_4_validation,0.441573,0.416988,0.462440,0.440352,0.507376,0.418560,0.554921,0.414091,0.445502,...,0.512316,0.481486,0.478083,0.419712,0.405483,0.457199,0.418430,0.457610,0.443268,0.499900
4,FOODS_1_001_TX_1_validation,0.069039,0.080348,0.074539,0.066957,0.064485,0.000000,0.070984,0.633484,0.511664,...,0.413691,0.437997,0.532788,0.426455,0.378689,0.371107,0.322752,0.354773,0.456160,0.457453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_validation,0.244731,0.207953,0.235976,0.230085,0.273229,0.344736,0.311665,0.234906,0.229433,...,0.267963,0.325624,0.319277,0.232865,0.203993,0.221699,0.220743,0.264876,0.302511,0.301994
30486,HOUSEHOLD_2_516_TX_3_validation,0.199098,0.195936,0.177368,0.188009,0.203327,0.231877,0.188973,0.183677,0.170873,...,0.183265,0.202961,0.200686,0.157450,0.168777,0.160447,0.170854,0.181794,0.207355,0.199485
30487,HOUSEHOLD_2_516_WI_1_validation,0.084894,0.081096,0.082114,0.086999,0.116116,0.120130,0.100854,0.091252,0.092088,...,0.122999,0.144647,0.122855,0.092204,0.090090,0.091197,0.096625,0.125388,0.154478,0.131501
30488,HOUSEHOLD_2_516_WI_2_validation,0.072411,0.069667,0.040580,0.073047,0.081257,0.148608,0.095028,0.066746,0.082574,...,0.056194,0.106998,0.064933,0.061898,0.099711,0.072574,0.069796,0.077769,0.102354,0.089733


In [220]:
ensemble.to_csv('data/submission/netflix_ensemble.csv', index=False)